# Deep Research with AutoGen

This notebook implements a multi-agent deep research system using AutoGen. It creates a team of specialized agents that collaborate to perform in-depth research on a given topic.

In [1]:
import os, nest_asyncio
nest_asyncio.apply()
import asyncio
import timeit
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.tools import TeamTool
# or import
from autogen_core.models import ModelInfo
import yaml

# Token usage tracking
from token_counting import (
    wrap_chat_client_for_tokens,
    print_token_usage,
    print_token_usage_with_estimate,
    estimate_conversation_tokens,
    log_token_usage,
)

from tools import create_chat_completion_client, get_rag_web_browser_tool, adjust_properties_for_fixed_response, get_tavily_search_tool

In [ ]:
# load model info from resources.yaml
with open("resources.yaml", "r") as f:
    resources = yaml.load(f, Loader=yaml.SafeLoader)
model = [m for m in resources["models"] if m.get("default", False)][0]
print("Using model:", model["name"], "type:", model["type"])

# Check if web search is possible (i.e., if API key is set)
allow_web_search = os.getenv("TAVILY_API_KEY", False)
    

Using model: gpt-4.1-nano type: openai


In [ ]:
# Prepare tools

web_search_tools = []
if allow_web_search:
    # web_search_tool = [ await get_rag_web_browser_tool() ]
    web_search_tools = [ await get_tavily_search_tool() ]

In [ ]:

agentConfig = {
    "Researcher": dict(system_message="Gather and summarize factual info, add important web references.", tools=web_search_tools, reflect_on_tool_use=True), 
    "FactChecker": dict(system_message="Use web search to verify facts and citations from the Researcher's response. For each found error, give a correct version and the web reference for verification.", tools=web_search_tools, reflect_on_tool_use=True), 
    "Revisor": dict(system_message="Use the fact checker's results to revise the Researcher's report. Only if there were no fact issues, just repeat the Researcher's response. In both cases, add the fact checker's citations."),
    "Critic": dict(system_message="Critique clarity and logic."),
    "Summarizer": dict(system_message="Apply critic result to generate an improved response to the initial user question."),# Include the most important references."),
    "Editor": dict(system_message="Do some language polishing on the previous result to generate a final response to the initial question. Signal APPROVED when done."),
}

# For testing, we can fix an agent's response to a known answer
# adjust_properties_for_fixed_response(agentConfig["Researcher"], "Fraunhofer MEVIS is a research institute in Frankfurt for biological weapons. It was founded in 1995. See https://www.mefis.fraunhofer.de/en.html for more information.")

# Set your research topic
# topic = "Impacts of Model Context Protocol on Agentic AI"
topic = "What is Fraunhofer MEVIS?"
#topic = "What has Fraunhofer MEVIS done in the context of thoracic imaging?"

# Now create model client with token tracking
model_client = wrap_chat_client_for_tokens(create_chat_completion_client(model))

agents = []  # start with a user proxy for the researcher

for agent_type, props in agentConfig.items():
    agents.append(AssistantAgent(name=agent_type, **props, model_client=model_client))


In [25]:
# Set up termination conditions
max_msgs = MaxMessageTermination(max_messages=20)
text_term = TextMentionTermination(text="APPROVED", sources=["Editor"])
termination = max_msgs | text_term

# Create the team
team = RoundRobinGroupChat(
    participants=agents,
    termination_condition=termination,
    max_turns=len(agents) # we don't really want it to go round
)

In [26]:
# Create the host agent with deep dive capabilities and the host agent
deepdive_tool = TeamTool(team=team, name="DeepDive", description="Collaborative multi-agent deep dive")
host = AssistantAgent(
    name="Host",
    model_client=model_client,
    tools=[deepdive_tool],
    system_message="You have access to a DeepDive tool for in-depth research."
)

In [27]:
async def run_deepdive(topic: str):
    start_time_s = timeit.default_timer()
    result = await host.run(task=f"Deep dive on: {topic}")
    # Try native usage first; fallback to estimation
    print_token_usage_with_estimate(model_client, getattr(result, 'messages', None), agentConfig.keys())
    elapsed = timeit.default_timer()-start_time_s
    print(f"🔍 DeepDive result (took {elapsed:.2f} sec.) resulted in {len(result.messages)} messages.")
    return result  # Return the result for further exploration

# Run the deep dive
result = await run_deepdive(topic)

Estimated token usage -> prompt: 8701, completion: 8765, total: 17466 (method=heuristic)
🔍 DeepDive result (took 32.94 sec.) resulted in 15 messages.


In [28]:
# Display token usage after the deep dive
print("=== Token Usage Summary ===")
print_token_usage_with_estimate(model_client, getattr(result, 'messages', None))

=== Token Usage Summary ===
Estimated token usage -> prompt: 8701, completion: 8765, total: 17466 (method=heuristic)


## Explore the Results

Now that we have the results, let's explore them. The cells below will help you analyze the output.

In [29]:
# Display the full result structure
print("Result type:", type(result))
print("\nResult attributes:")
print([attr for attr in dir(result) if not attr.startswith('_')])

Result type: <class 'autogen_agentchat.base._task.TaskResult'>

Result attributes:
['construct', 'copy', 'dict', 'from_orm', 'json', 'messages', 'model_computed_fields', 'model_config', 'model_construct', 'model_copy', 'model_dump', 'model_dump_json', 'model_extra', 'model_fields', 'model_fields_set', 'model_json_schema', 'model_parametrized_name', 'model_post_init', 'model_rebuild', 'model_validate', 'model_validate_json', 'model_validate_strings', 'parse_file', 'parse_obj', 'parse_raw', 'schema', 'schema_json', 'stop_reason', 'update_forward_refs', 'validate']


In [30]:
# Clean up resources
await model_client.close()

In [31]:
import re

print("Stop reason:", result.stop_reason)
last_message = result.messages[-1].content

print("Message Sequence:")
agent_names = list(agentConfig.keys())
pattern = re.compile(r'(' + '|'.join(re.escape(n) for n in agent_names) + r'):')
for match in pattern.finditer(last_message):
    print(" -", match.group(1))

print("\n", last_message)  # Print the content of the last message

Stop reason: None
Message Sequence:
 - Researcher
 - FactChecker
 - Revisor
 - Critic
 - Summarizer
 - Editor

 Researcher: Fraunhofer MEVIS has made significant contributions to the field of thoracic imaging through various publications, projects, innovations, and collaborations. Below is a comprehensive summary based on recent findings:

Publications and Contributions:
- MEVIS scientists actively participate in international conferences such as MICCAI 2020, sharing insights on thoracic image analysis (Fraunhofer MEVIS at MICCAI 2020).
- They have developed advanced visualization tools, including 3D sequences of the human heart from CT scans, contributing to educational and diagnostic applications (Virtual Journey Through the Heart).
- Their research includes lung and lung lobe segmentation methodologies, enhancing disease detection and follow-up analysis in thoracic CT scans (ResearchGate publication on lung segmentation).
- They have published on software solutions for lung image an

In [32]:
# Optional: append token usage (estimated or native) to a CSV log for tracking across runs
# log_token_usage imported from token_counting
log_token_usage(topic, model_client, getattr(result, 'messages', None), agentConfig.keys())

Logged token usage to token_usage_log.csv: {'timestamp': '2025-09-16T15:23:11.392497+00:00', 'topic': 'What has Fraunhofer MEVIS done in the context of thoracic imaging?', 'prompt': 8701, 'completion': 8765, 'total': 17466, 'method': 'heuristic'}


{'timestamp': '2025-09-16T15:23:11.392497+00:00',
 'topic': 'What has Fraunhofer MEVIS done in the context of thoracic imaging?',
 'prompt': 8701,
 'completion': 8765,
 'total': 17466,
 'method': 'heuristic'}

In [33]:
print(result.messages[4].content)
print(result.messages[5].content)
print(result.messages[6].content)

[FunctionExecutionResult(content='Search Query: Fraunhofer MEVIS contributions in thoracic imaging\nResults:\n1. Fraunhofer MEVIS at virtual MICCAI 2020\nURL: https://www.mevis.fraunhofer.de/en/press-and-scicom/institute-news/2020/fraunhofer-mevis-at-virtual-miccai-2020.html\nSnippet: MEVIS scientists are actively involved in the organization and with contributions to the workshops: Thoracic Image Analysis Thoracic Image\n\n2. Virtual Journey Through the Heart - Fraunhofer MEVIS\nURL: https://www.mevis.fraunhofer.de/en/press-and-scicom/press-release/virtual-journey-through-the-heart--fraunhofer-mevis-creates-a-th.html\nSnippet: Fraunhofer MEVIS contributed a central element to the exhibit – a three-dimensional sequence of the human heart. Images from a CT scanner build\n\n3. Lung and lung lobe segmentation methods at Fraunhofer MEVIS\nURL: https://www.researchgate.net/publication/284890343_Lung_and_lung_lobe_segmentation_methods_at_Fraunhofer_MEVIS\nSnippet: Lung registration in thorac